In [ ]:
!pip install transformers

Here, I will take the uncleaned articles from the csv file and extract the dates from them. I will then create a new dataframe which will extract the date and remove articles with incorrect or unknown dates. I will also extract the title of the articles as well.

In [1]:
%load_ext autoreload
%autoreload 2
from google.colab import drive
drive.mount('/content/drive')

import os
import sys
GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'ECMA 31330 Final Project'
GOOGLE_DRIVE_PATH = os.path.join('drive', 'MyDrive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))
sys.path.append(GOOGLE_DRIVE_PATH)

Mounted at /content/drive
['Articles', 'SentimentScores', 'Aakash Gupta Final Project Presentation.gslides', 'Sp500Stocks.csv', 'SPY2015.csv', 'SPY2020.csv', 'SPY-2015_2020.csv', 'DataCleaningPipeline.ipynb', 'SentimentAnalysisPipeline.ipynb', 'SimpleGARCHModel.ipynb', 'LassoModel.ipynb', 'GARCHFittingPipeline.ipynb', 'NeuralNet.ipynb']


In [2]:
import pandas as pd
import numpy as np
import pandas as pd
import transformers
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import torch
import datetime
import yfinance as yf
from datetime import datetime

In [ ]:
#articles2015 = pd.read_csv(GOOGLE_DRIVE_PATH + "/Articles/articles2015.csv")
# articles2016 = pd.read_csv("Articles/articles2016.csv")
# articles2017 = pd.read_csv("Articles/articles2017.csv")
# articles2018 = pd.read_csv("Articles/articles2018.csv")
# articles2019 = pd.read_csv("Articles/articles2019.csv")
articles2015 = pd.read_csv(GOOGLE_DRIVE_PATH + "/Articles/articles2016.csv", header = None)
articles2015.columns = ["article_id", "article_url", "comment_count", "year"]

In [ ]:
def date_collector(article_url, year):
    date = str.split(article_url, '/')
    for i in range(len(date)):
        if (date[i] == year):
          prospective_date = date[i:i+3]
          result = all(ele.isdigit() for ele in prospective_date)
          if (result):
            return date[i:i+3]
          else:
            return None
    return None
def title_collector(article_url):
    title = article_url.split("/")[::-1][0]
    if (title == ""):
      title = article_url.split("/")[::-1][1]
    title = title.replace("-", " ")
    title = title.replace(".html", "")
    return title
def construct_dates(date):
    if (date != None):
        return date[2] + "/" + date[1] + "/" + date[0]
    else:
        return "01/01/1975"

In [ ]:
dates = articles2015['article_url'].apply(date_collector,args=("2016",))
titles = articles2015['article_url'].apply(title_collector, 0)
articles2015['date'] = dates.apply(construct_dates,0)
articles2015['title'] = titles

<ipython-input-45-90416fe88d3a>:2: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  titles = articles2015['article_url'].apply(title_collector, 0)
<ipython-input-45-90416fe88d3a>:3: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  articles2015['date'] = dates.apply(construct_dates,0)


In [ ]:
number_of_bad_articles = len(articles2015.groupby("date")['article_id'].apply(list).reset_index()['article_id'][0])
total_articles = len(articles2015)
percentage_bad_articles = number_of_bad_articles/total_articles * 100
print(str(percentage_bad_articles) + "% of articles do not have the correct date and will be removed from the dataset")

2.4881277881709596% of articles do not have the correct date and will be removed from the dataset


In [ ]:
good_ids = articles2015.groupby("date")['article_id'].apply(list).reset_index()['article_id'][1:366]
good_ids = sum(good_ids, [])

In [ ]:
good_articles_2015 = articles2015[articles2015['article_id'].isin(good_ids)]

In [ ]:
good_articles_2015['datetimeobject'] = good_articles_2015['date'].astype('datetime64[ns]')
good_articles_2015 = good_articles_2015.sort_values('datetimeobject')

<ipython-input-49-e54e23a87cf5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  good_articles_2015['datetimeobject'] = good_articles_2015['date'].astype('datetime64[ns]')


In [ ]:
good_articles_2015.to_csv(GOOGLE_DRIVE_PATH + "/Articles/cleaned2016articles.csv")

In [ ]:
good_articles_2015 = pd.read_csv(GOOGLE_DRIVE_PATH + "/Articles/cleaned2016articles.csv")

In [ ]:
good_articles_2015.groupby('datetimeobject').count()

,Unnamed: 0,article_id,article_url,comment_count,year,date,title
datetimeobject,,,,,,,
2016-01-01,51,51,51,51,51,51,51
2016-01-02,176,176,176,176,176,176,176
2016-01-03,219,219,219,219,219,219,219
2016-01-04,218,218,218,218,218,218,218
2016-01-05,256,256,256,256,256,256,256
...,...,...,...,...,...,...,...
2016-12-26,88,88,88,88,88,88,88
2016-12-27,140,140,140,140,140,140,140
2016-12-28,153,153,153,153,153,153,153


#TODO:
We are going to filter the article titles that are only related to finance, business, or anything item that could affect the stock market

In [ ]:
keywords_title = ['lockdown','covid','virus','terrorist', 'dispute', 'fears', 'fear', 'crash', 'business', 'finance', 'war', 'automation',
                  'stocks', 'conflict', 'sanction', "sanctions", "leaders", "crisis", "tension", "hikes", "panic", "recession", "tumble", "turmoil",
                  "uncertainty", "volatility", "inflation", "layoff", "decline", "bankruptcy", "covid-19", "China", "restrictions", "tariffs"]
keywords_url = ['business', 'finance', 'world', 'politics', 'your-money']
#filtered_titles = titles[titles.str.contains('|'.join(keywords), case=False, na=False)]

# Print results
#print(filtered_titles)
filtered_urls = good_articles_2015[good_articles_2015['article_url'].str.contains('|'.join(keywords_url), case=False, na=False)]
filtered_titles = filtered_urls[filtered_urls['title'].str.contains('|'.join(keywords_title), case=False, na=False)]
filtered_titles.to_csv(GOOGLE_DRIVE_PATH + "/Articles/filtered2016articles.csv")

,Unnamed: 0,article_id,article_url,comment_count,year,date,title,datetimeobject
176,6820,6821,https://www.nytimes.com/2015/02/01/world/europ...,0,2015,01/02/2015,stig bergling a cold war spy known for his esc...,2015-01-02
237,6868,6869,https://www.nytimes.com/2015/02/01/world/asia/...,0,2015,01/02/2015,clashes intensify between armenia and azerbaij...,2015-01-02
322,13301,13302,https://www.nytimes.com/2015/03/01/business/en...,0,2015,01/03/2015,britain wary of sanctions adds pressure to kee...,2015-01-03
340,13273,13274,https://www.nytimes.com/2015/03/01/world/us-se...,0,2015,01/03/2015,us seeks to deport bosnians over war crimes,2015-01-03
386,13455,13456,https://bits.blogs.nytimes.com/2015/03/01/atts...,5,2015,01/03/2015,atts chief of mobile and business explains the...,2015-01-03
...,...,...,...,...,...,...,...,...
74808,78580,78581,https://www.nytimes.com/politics/first-draft/2...,0,2015,31/12/2015,top ben carson aides quit amid campaign turmoil,2015-12-31
74849,78367,78368,https://www.nytimes.com/2015/12/31/world/europ...,0,2015,31/12/2015,russia putin turkey sanctions,2015-12-31
74851,78369,78370,https://www.nytimes.com/2015/12/31/business/de...,0,2015,31/12/2015,icahn wins bidding war for pep boys,2015-12-31
74852,78376,78377,https://www.nytimes.com/2015/12/31/business/ka...,0,2015,31/12/2015,kalobios bankruptcy martin shkreli chagas disease,2015-12-31
